In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from collections import Counter
from performance_helper import get_performance_scores, get_finetuned_scores
from table_helper import create_tabular
from table_boldener import bolden_table_max_values_with_hline
import io
import itertools
import shutil

from dataloader import DataLoader
dataloader = DataLoader()

import re

N_SEEDS = 5
TASK_FOR_TABLE = "ASQP"

tasks = ["asqp", "tasd"]
datasets = ["rest15", "rest16", "gerest", "flightabsa", "hotels"]
dataset_types = ["test"]
models = ["gemma3:4b", "gemma3:27b"]
seeds = [0, 1, 2, 3, 4]
modes = ["label"] # "label"
n_shots = [0, 10, 20, 30, 40, 50]

# combinations = itertools.product(tasks, datasets, dataset_types, models, seeds, modes, n_shots)

# for comb in combinations:
#     source_path = f'../generations/zeroshot/{comb[0]}_rest16_{comb[2]}_{comb[3]}_{comb[4]}_{comb[5]}_{comb[6]}.json'
#     destination_path = f'../generations/zeroshot/{comb[0]}_gerest_{comb[2]}_{comb[3]}_{comb[4]}_{comb[5]}_{comb[6]}.json'
    
#     if os.path.exists(destination_path) == False:
#         try:
#             shutil.copy(source_path, destination_path)
#             print(f"Kopiert: {source_path} -> {destination_path}")
#         except Exception as e:
#             print(f"Fehler beim Kopieren von {source_path} nach {destination_path}: {e}")
#     else:
#         print(f"Quelle existiert nicht: {source_path}")

Wichtig TO-DO: Prüfen, ob 5-split gleich ist wie bei dataloader!!!
Wichtig TO-DO: Prüfen, jemals in einem datensatz ein triplet oder quadruple bei einem beispiel doppelt vorkam

In [2]:
get_performance_scores("tasd", "rest16", "gemma3:27b", "label", "test", n_shots=50, self_consistency=False, minimum_appearance=3, n_seeds=N_SEEDS)

{'precision': 62.80889864875758,
 'recall': 63.515715948777654,
 'f1': 63.159836635765316,
 'TP': 545.6,
 'FP': 323.0,
 'FN': 313.8}

In [3]:
get_finetuned_scores("tasd", "rest16", "mvp")

{'f1': 72.76, 'precision': '-', 'recall': '-'}

In [4]:
def limit_decimal_points(num_str, n_decimal=2):
    # Convert the string to a float and format it with the specified number of decimal places
    return f"{float(num_str):.{n_decimal}f}"

## Create Table

In [5]:
raw_task_to_formatted = {"asqp": "ASQP", "tasd": "TASD"}
format_task_to_raw = {"ASQP": "asqp", "TASD": "tasd"}

raw_dataset_to_formatted = {"rest16": "Rest16", "rest15": "Rest15", "flightabsa": "FlightABSA", "gerest": "GERest", "hotels": "OATS Hotels"}
format_dataset_to_raw = {"Rest16": "rest16", "Rest15": "rest15", "FlightABSA": "flightabsa", "GERest": "gerest", "OATS Hotels": "hotels"}

raw_llm_to_formatted = {"gemma3:27b": "Gemma-3-27B", "llama3.1:70b-instruct-q4_0": "Llama-3-27B", "gemma3:4b": "Gemma-3-4B"}
format_llm_to_raw = {"Gemma-3-27B": "gemma3:27b", "Llama-3-70B": "llama3.1:70b-instruct-q4_0", "Gemma-3-4B": "gemma3:4b"}

data = {"Method": ["Gemma-3-4B"] * 12 + ["Gemma-3-27B"] * 12, "#Few-Shot Examples": [0, 10, 20, 30, 40, 50] * 4, "Prompting Strategy": ["-"] * 6 + ["SC"] * 6 + ["-"] * 6 + ["SC"] * 6 }


for dataset in datasets:
       for metric in ["f1", "precision", "recall"]:
            data[f"{dataset}_{metric}"] = [get_performance_scores(format_task_to_raw[TASK_FOR_TABLE], 
                                                  dataset, 
                                                  model,
                                                  "label", 
                                                  "test", # evaluation target 
                                                  data["#Few-Shot Examples"][i],
                                                  self_consistency=sc, 
                                                  minimum_appearance=3, n_seeds=N_SEEDS, evaluation_type="quad")[metric] for model in models for sc in [False, True] for i in range(6)]

In [ ]:
rows = [{"name": "Method", "type": "string", "struct": True}, 
        {"name": "Prompting Strategy", "type": "string", "struct": True},
        {"name": "#Few-Shot Examples", "type": "string", "struct": False},]
for dataset in datasets:
    for metric in ["f1", "precision", "recall"]:
        rows += [{"name": f"{dataset}_{metric}", "type": "double", "struct": False, "decimal_place": 2}]


output = io.StringIO()
sys.stdout = output
create_tabular(rows=rows, data=data)
printed_output = output.getvalue()
sys.stdout = sys.__stdout__
output.close()
print(printed_output)
print(bolden_table_max_values_with_hline(printed_output, [0, 1, 2]))

In [7]:
raw_task_to_formatted = {"asqp": "ASQP", "tasd": "TASD"}
format_task_to_raw = {"ASQP": "asqp", "TASD": "tasd"}

raw_dataset_to_formatted = {"rest16": "Rest16", "rest15": "Rest15", "flightabsa": "FlightABSA", "gerest": "GERest", "hotels": "OATS Hotels"}
format_dataset_to_raw = {"Rest16": "rest16", "Rest15": "rest15", "FlightABSA": "flightabsa", "GERest": "gerest", "OATS Hotels": "hotels"}

raw_llm_to_formatted = {"gemma3:27b": "Gemma-3-27B", "llama3.1:70b-instruct-q4_0": "Llama-3-27B"}
format_llm_to_raw = {"Gemma-3-27B": "gemma3:27b", "Llama-3-70B": "llama3.1:70b-instruct-q4_0"}

data = {"Method": ["MVP \citep{gou2023mvp}"] * 7 + ["DLO \citep{hu2022improving}"] * 7 + ["Paraphrase \citep{zhang2021aspect}"] * 7, "#Few-Shot Examples": [10, 20, 30, 40, 50, 800, "Full"] * 3, "Prompting Strategy": ["-"] * 21 }

for dataset in datasets:
       for metric in ["f1", "precision", "recall"]:
            data[f"{dataset}_{metric}"] = [get_finetuned_scores(format_task_to_raw[TASK_FOR_TABLE], 
                                                  dataset, 
                                                  model,
                                                  n_shots=fs)[metric] for model in ["mvp", "paraphrase", "dlo"] for fs in [10, 20, 30, 40, 50, 800, None]]

\hline
\arrayrulecolor{gray}\cline{2-18}\arrayrulecolor{black}
\textbf{\multirow{12}{*}{Gemma-3-4B}} & \textbf{\multirow{6}{*}{-}} & 0 & 13.15 & 14.13 & 12.30 & 17.02 & 18.33 & 15.89 & 17.02 & 18.33 & 15.89 & 33.43 & 36.09 & 31.15 & 24.92 & 29.40 & 21.66 \\
\textbf{} & \textbf{} & 10 & 19.77 & 19.05 & 20.55 & 25.06 & 24.12 & 26.08 & 25.06 & 24.12 & 26.08 & 22.31 & 22.50 & 22.14 & 20.22 & 21.74 & 18.89 \\
\textbf{} & \textbf{} & 20 & 22.55 & 21.98 & 23.14 & 29.54 & 27.94 & 31.34 & 29.54 & 27.94 & 31.34 & 25.70 & 26.43 & 25.02 & 24.15 & 25.69 & 22.79 \\
\textbf{} & \textbf{} & 30 & 22.66 & 22.37 & 22.97 & 27.04 & 25.73 & 28.49 & 27.04 & 25.73 & 28.49 & 25.86 & 26.49 & 25.25 & 23.86 & 25.79 & 22.21 \\
\textbf{} & \textbf{} & 40 & 27.19 & 27.14 & 27.25 & 32.82 & 31.47 & 34.29 & 32.82 & 31.47 & 34.29 & 30.21 & 30.41 & 30.03 & 27.96 & 30.53 & 25.80 \\
\textbf{} & \textbf{} & 50 & 26.29 & 26.09 & 26.49 & 34.43 & 33.05 & 35.92 & 34.43 & 33.05 & 35.92 & 29.90 & 30.25 & 29.56 & 29.75 & 32.74 & 2

In [8]:
rows = [{"name": "Method", "type": "string", "struct": True}, 
        {"name": "Prompting Strategy", "type": "string", "struct": True},
        {"name": "#Few-Shot Examples", "type": "string", "struct": False},]
for dataset in datasets:
    for metric in ["f1", "precision", "recall"]:
        rows += [{"name": f"{dataset}_{metric}", "type": "double", "struct": False, "decimal_place": 2}]


output = io.StringIO()
sys.stdout = output
create_tabular(rows=rows, data=data)
printed_output = output.getvalue()
sys.stdout = sys.__stdout__
output.close()
print(printed_output)
print(bolden_table_max_values_with_hline(printed_output, [0, 1, 2]))